# DSE 309 Final Project Sambhav Agrawal 19264 Data Science and Engineering


# Brain Tumor Detection and Classification using Convolutional Neutral Networks

<center><img src="https://media.istockphoto.com/photos/brain-cancer-picture-id174927353?k=6&m=174927353&s=612x612&w=0&h=P3EzH6J-haFHzmKQ8RzkfotD_lsygXzNUL72Dg3pfK0=",height='300',width='600'></center>

---

# Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']

from IPython.display import display,clear_output
from warnings import filterwarnings
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

---

---

# Data Preperation

In [2]:
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

I have appended all the images from the  directories into a Python list and then transformed them into numpy arrays.

In [3]:
X_train = []
y_train = []
X_test = []
y_test = []
image_size = 128
for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_test.append(img)
        y_test.append(i)
        
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [4]:
k=0
fig, ax = plt.subplots(1,4,figsize=(10,10))
fig.text(s='Sample Image From Each Label',size=10,fontweight='bold',
             fontname='monospace',y=0.64,x=0.4,alpha=0.8)
for i in labels:
    j=0
    while True :
        if y_train[j]==i:
            ax[k].imshow(X_train[j])
            ax[k].set_title(y_train[j])
            ax[k].axis('off')
            k+=1
            break
        j+=1

In [5]:
X_train, y_train = shuffle(X_train,y_train, random_state=123)

In [ ]:
X_test.shape

In [6]:
X_train.shape

Dividing the dataset into **Training** and **Testing** sets.

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=123)

Performing **One Hot Encoding** on the labels after converting it into numerical values:

In [8]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

---

# CNN MODEL

In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50


In [10]:
resnet = ResNet50(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [11]:
model = resnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=resnet.input, outputs = model)

In [12]:
model.summary()

We finally compile our model.

In [13]:
model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

In [14]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("resnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 1, min_delta = 0.001,
                              mode='auto',verbose=1)

In [15]:
model.save('resnet.h5')

---

# Training The Model

In [16]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =20, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

In [17]:
filterwarnings('ignore')

epochs = [i for i in range(20)]
fig, ax = plt.subplots(1,2,figsize=(16,8))
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

fig.text(s='Epochs vs. Training and Validation Accuracy and Loss',size=20,fontweight='bold',
             color=colors_dark[1],y=1,x=0.3,alpha=0.8)

sns.despine()
ax[0].plot(epochs, train_acc, marker='o',markerfacecolor=colors_green[2],color=colors_green[3],
           label = 'Training Accuracy')
ax[0].plot(epochs, val_acc, marker='o',markerfacecolor=colors_red[2],color=colors_red[3],
           label = 'Validation Accuracy')
ax[0].legend(frameon=False)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

sns.despine()
ax[1].plot(epochs, train_loss, marker='o',markerfacecolor=colors_green[2],color=colors_green[3],
           label ='Training Loss')
ax[1].plot(epochs, val_loss, marker='o',markerfacecolor=colors_red[2],color=colors_red[3],
           label = 'Validation Loss')
ax[1].legend(frameon=False)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Training & Validation Loss')

fig.show()

---

# Prediction

In [18]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

---

# Evaluation

In this the following convention is used as shown below: <br>
0 - Glioma Tumor<br>
1 - No Tumor<br>
2 - Meningioma Tumor<br>
3 - Pituitary Tumor<br>

In [19]:
print(classification_report(y_test_new,pred))

In [20]:
fig,ax=plt.subplots(1,1,figsize=(14,7))
sns.heatmap(confusion_matrix(y_test_new,pred),ax=ax,xticklabels=labels,yticklabels=labels,annot=True,
           alpha=0.7,linewidths=2,linecolor=colors_dark[3])
fig.text(s='Heatmap of the Confusion Matrix',size=18,fontweight='bold',
             color=colors_red[1],y=1,x=0.3,alpha=0.8)

plt.show()

---

In [21]:
import colorama
from colorama import Fore



for i in range(100):
    if pred[i] == y_test_new[i] == 0:
        print(Fore.GREEN + 'Correct Prediction Glioma')
    
    elif pred[i] == y_test_new[i] == 1:
        print(Fore.GREEN + 'Correct Prediction No Tumor')
    
    elif pred[i] == y_test_new[i] == 2:
        print(Fore.GREEN + 'Correct Prediction Meningioma ')
    
    elif pred[i] == y_test_new[i] == 3:
        print(Fore.GREEN + 'Correct Prediction Pituitary ')
    
    else:
        print(Fore.RED + 'InCorrect Prediction')

---

In [22]:
score = model.evaluate(X_test, y_test, verbose = 1)
score[1]

print('The above ResNet50 model gave an accuracy of around 95% which is quite nice. I have modified the ResNet50 model further to get even more accuracy.')

---